In [2]:
import gzip
import json

def messages_parse(data):
    data_with_no_binary = str(data).split("'")[1:-1][0]
    data_list =[i for i in data_with_no_binary.split('\\n') if i != '']
    messages_data = [json.loads(D) for D in data_list]
    return messages_data 

def messages_read(file="../data/coinbase_BTC-USD_20_10_06_000000-010000.json.gz"):
    file_object = gzip.open(file, "r")
    data = file_object.read()
    messages_data = messages_parse(data)
    return messages_data 


def snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json"):
    snaphsot_data = json.loads(open(file,'r').read())
    return snaphsot_data

messages_data = messages_read()
initial_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json")
final_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_15.json")

In [8]:
def messages_filter(messages_data,initial_clob,final_clob):
    messages_data_filtered = [message_dict for message_dict in messages_data if message_dict['sequence'] >= initial_clob['sequence'] and message_dict['sequence'] <= final_clob['sequence']]
    return messages_data_filtered
                                                                                        
messages_data_filtered = messages_filter(messages_data,initial_clob,final_clob)

                                                                                        
                                                                                    

In [9]:
print (len(messages_data))
print (len(messages_data_filtered))

1086672
252618


In [4]:
print (messages_data[0])
print (initial_clob.keys())
print (final_clob.keys())

{'type': 'done', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:00.021717Z', 'sequence': 16712803594, 'order_id': '0e612005-8b3c-4b34-a2db-eadc20afc95e', 'reason': 'canceled', 'price': '10802.46', 'remaining_size': '0.25'}
dict_keys(['bids', 'asks', 'sequence'])
dict_keys(['bids', 'asks', 'sequence'])


In [27]:
messages_data_filtered[100]

{'type': 'open',
 'side': 'buy',
 'product_id': 'BTC-USD',
 'time': '2020-10-06T00:00:05.860994Z',
 'sequence': 16712805003,
 'price': '10778.91',
 'order_id': '5cee9745-3cfa-46ee-b48d-31511a163ae7',
 'remaining_size': '1.31839767'}

In [24]:

filter_order_id = lambda array,order_id: [D for D in array if 'order_id' in D.keys() and D['order_id'] ==order_id]
filter_order_id(messages_data_filtered,"19b9d70e-8821-4846-ba72-ce5eeba2feaa")


[]

In [23]:
filter_order_id = lambda clob,asks_or_bids,order_id: [subli for subli in clob[asks_or_bids] if subli[2] == order_id]
filter_order_id(final_clob,'asks','d98e69bd-aa7c-40ec-8c34-5087c20b86bd')

[]

In [16]:
len([message_dict for message_dict in messages_data_filtered if 'order_id' not in message_dict.keys()])

1077

In [17]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'